In [1]:
import os, time, shutil, itertools, json
import numpy as np
import scipy.stats
from collections import defaultdict
from tqdm import tqdm, tqdm_notebook


def crawl_directory(dirname):
    """ Walk a nested directory to get all filename ending in a pattern """
    for path, subdirs, files in os.walk(dirname):
        for name in files:
            if not name.endswith('.DS_Store'):
                yield os.path.join(path, name)


def remove_empty_dirs(path):
    for root, dirnames, filenames in os.walk(path, topdown=False):
        for dirname in dirnames:
            remove_empty_dir(os.path.realpath(os.path.join(root, dirname)))


def remove_empty_dir(path):
    try:
        os.rmdir(path)
    except OSError:
        pass
    

def nested_pickle_dict():
    """ Picklable defaultdict nested dictionaries """
    return defaultdict(nested_pickle_dict)


def format_e(n):
    a = '%E' % n
    return (a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]).lower()


def mean_confidence_interval(data, axis=1, confidence=0.95):
    try:
        n = data.shape[axis]
    except IndexError:
        axis = 0
        n = data.shape[axis]

    mu, std = np.nanmean(data, axis=axis), scipy.stats.sem(data, axis=axis, nan_policy='omit')
    h = np.ma.getdata(std) * scipy.stats.t.ppf((1 + confidence) / 2., n-1)       

    return mu, h, mu-h, mu+h

In [ ]:
""" Get best performance according to hyperparameter mean minimum """

dirname = '/Users/sob/Desktop/gan_results/hypertuning/multivariate/'

# Same for all..
resultnames = [i for i in os.listdir(dirname + '16_dims_1000_samples/trial_1') if i != '.DS_Store']
tuningnames = [i for i in os.listdir(dirname) if i != '.DS_Store']

for t in tqdm_notebook(tuningnames):
    
    print(t)
    
    best_path = '/Users/sob/Desktop/gan_results/best_hyper/multivariate/{0}/'.format(t)
    global_optimal = nested_pickle_dict()

    # Initialize best dictionary    
    for name in tqdm_notebook(resultnames):

        optimal = nested_pickle_dict()
        results = []

        # Load in the results from each trial
        for trial in range(1, 21):
            path = '/Users/sob/Desktop/gan_results/hypertuning/multivariate/{0}/trial_{1}/{2}'.format(t, trial, name)
            with open(path, 'r') as f:
                data = json.load(f)

            results.append(data)

        # Go through each one and append the results
        for result in results:
            for model, distributions in result.items():
                for distribution, metrics in distributions.items():
                    for metric, values in metrics.items():
                        if metric in ["LR", "HDIM", 'GLoss', 'DLoss', "BSIZE", "Energy-Distance"]:
                            continue
                        else:
                            # If metric is seen for the first time, initialize it
                            if 'values' not in optimal[model][distribution][metric]:
                                optimal[model][distribution][metric]["values"] = []

                            # Otherwise, compare it the presently considered value
                            optimal[model][distribution][metric]["values"].append(values)

        for model, distributions in result.items():
            for distribution, metrics in distributions.items():
                for metric, values in metrics.items():
                    if metric in ["LR", "HDIM", 'GLoss', 'DLoss', "BSIZE", "Energy-Distance"]:
                        continue
                    else:
                        data_min = np.nanmean(np.nanmin(np.array(optimal[model][distribution][metric]["values"]), axis=1))
                        
                        # Init global optimal
                        if 'best' not in global_optimal[model][distribution][metric]:
                            global_optimal[model][distribution][metric]['best'] = 1e10     

                        if data_min < global_optimal[model][distribution][metric]['best']:
#                             print(model, distribution, metric, '\n', global_optimal[model][distribution][metric]['best'], '-->', data_min, '\n')
                            global_optimal[model][distribution][metric]['best'] = data_min
                            global_optimal[model][distribution][metric]['parameters'] = [metrics["LR"], metrics["HDIM"], metrics["BSIZE"]]
                            global_optimal[model][distribution][metric]["values"] = optimal[model][distribution][metric]["values"]
            
                            mean, h, low, high = mean_confidence_interval(np.array(optimal[model][distribution][metric]["values"]))
                
                            global_optimal[model][distribution][metric]['low'] = list(low)
                            global_optimal[model][distribution][metric]['h'] = list(h)
                            global_optimal[model][distribution][metric]['mean'] = list(mean)
                            global_optimal[model][distribution][metric]['high'] = list(high)

    if not os.path.exists(best_path):
        os.makedirs(best_path)
    
    with open(best_path + 'data.json', 'w') as outfile:
        json.dump(global_optimal, outfile)


128_dims_100000_samples


/Users/sob/Desktop/gan_results/best_hyper/multivariate/128_dims_100000_samples/
128_dims_10000_samples


/Users/sob/Desktop/gan_results/best_hyper/multivariate/128_dims_10000_samples/
128_dims_1000_samples


/Users/sob/Desktop/gan_results/best_hyper/multivariate/128_dims_1000_samples/
16_dims_100000_samples


In [ ]:
global_optimal[model][distribution]['KL-Divergence']["mean"]

In [ ]:
best_path = '../best/' + '/'.join(dirname.split('/')[-3:])
if not os.path.exists(best_path):
    os.makedirs(best_path)

files = os.listdir(dirname)
files = [f for f in files if f != '.DS_Store']
for idx, f in tqdm.tqdm_notebook(enumerate(files)):

    optimal = get_best_performance_mnist(dirname + f + '/')
    if len(os.listdir(dirname + f + '/')) < 60:
        print(f, len(os.listdir(dirname + f + '/')))

    with open(best_path + '/trial_{0}.json'.format(idx+1), 'w') as outfile:
        json.dump(optimal, outfile)


In [ ]:
global_optimums[model][distribution]['Wasserstein-Distance']

In [ ]:
def get_best_performance_multivariate(mypath):
    """ For a trial, get the best performance for multivariate data according to any hyperparam """
    # Get path, files in path
    files = os.listdir(mypath)
    results = []

    # Read in the files
    for file in files:
        if file == '.DS_Store':
            continue
                            
        with open(mypath + file, 'r') as f:
            data = json.load(f)

        results.append(data)
        
    # Initialize best dictionary
    optimal = nested_pickle_dict()

    # Go through all models, distributionss, metrics, and record the best
    for result in results:
        for model, distributions in result.items():
            for distribution, metrics in distributions.items():
                for metric, values in metrics.items():
                    if metric not in ["LR", "HDIM", "BSIZE"]:

                        # If metric is seen for the first time, it is the best
                        if metric not in optimal[model][distribution]:
                            optimal[model][distribution][metric]["value"] = values
                            optimal[model][distribution][metric]["parameters"] = [metrics["LR"], metrics["HDIM"], metrics["BSIZE"]]

                        # Otherwise, compare it the presently considered value
                        elif min(optimal[model][distribution][metric]["value"]) > min(values):
                            optimal[model][distribution][metric]["value"] = values
                            optimal[model][distribution][metric]["parameters"] = [metrics["LR"], metrics["HDIM"], metrics["BSIZE"]]

    return optimal


def get_best_performance_mnist(*args):
    return get_best_performance_multivariate(*args)


def multivariate_hypertuning2best(dirname='/Users/sob/Desktop/gan_results/hypertuning/multivariate/64_dims_100000_samples/'):
    """ MOVE HYPERTUNING RESULTS TO BEST FOLDER """
    best_path = '../best/' + '/'.join(dirname.split('/')[-3:])
    if not os.path.exists(best_path):
        os.makedirs(best_path)

    files = os.listdir(dirname)
    files = [f for f in files if f != '.DS_Store']
    for idx, f in tqdm.tqdm_notebook(enumerate(files)):

        optimal = get_best_performance_mnist(dirname + f + '/')
        if len(os.listdir(dirname + f + '/')) < 60:
            print(f, len(os.listdir(dirname + f + '/')))

        with open(best_path + '/trial_{0}.json'.format(idx+1), 'w') as outfile:
            json.dump(optimal, outfile)


def merge_mixture(dirname):
    outdir = dirname
    for idx, file in enumerate(os.listdir(dirname)):

        if '.DS_Store' in file:
            continue

        for nest in crawl_directory(dirname + file):

            index = 1

            if 'dims' not in nest.split('/')[7]:
                outdir = '/'.join(nest.split('/')[:7] + nest.split('/')[8:9]) + '/'
            else:
                outdir = dirname
                
            # Initialize directory
            if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                os.makedirs(outdir + 'trial_{0}/'.format(index))

            try:
                shutil.move(nest, outdir + 'trial_{0}/'.format(index))
            except:
                extension = nest.split('/')[-1]
                while os.path.exists(outdir + 'trial_{0}/'.format(index) + extension):
                    index += 1

                if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                    os.makedirs(outdir + 'trial_{0}/'.format(index))

                shutil.move(nest, outdir + 'trial_{0}/'.format(index))

    remove_empty_dirs(dirname)
    
    
def merge_multivariate(dirname):
    outdir = dirname
    for idx, file in enumerate(os.listdir(dirname)):

        if '.DS_Store' in file:
            continue

        for nest in crawl_directory(dirname + file):

            index = 1

            if 'dims' not in nest.split('/')[6]:
                outdir = '/'.join(nest.split('/')[:7] + nest.split('/')[8:9]) + '/'
            else:
                # Uncomment the + for mixture
                outdir = dirname + nest.split('/')[6] + '/'
                
            # Initialize directory
            if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                os.makedirs(outdir + 'trial_{0}/'.format(index))

            try:
                shutil.move(nest, outdir + 'trial_{0}/'.format(index))
            except:
                extension = nest.split('/')[-1]
                while os.path.exists(outdir + 'trial_{0}/'.format(index) + extension):
                    index += 1

                if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                    os.makedirs(outdir + 'trial_{0}/'.format(index))

                shutil.move(nest, outdir + 'trial_{0}/'.format(index))

    remove_empty_dirs(dirname)
    
    
def merge_mnist(dirname):
    outdir = dirname
    for idx, file in enumerate(os.listdir(dirname)):

        if '.DS_Store' in file:
            continue

        for nest in crawl_directory(dirname + file):

            index = 1

            if 'dims' in nest.split('/')[5]:
                outdir = '/'.join(nest.split('/')[:6]) + '/'
            else:
                # Uncomment the + for mixture
                outdir = dirname + nest.split('/')[7] + '/'
    

            # Initialize directory
            if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                os.makedirs(outdir + 'trial_{0}/'.format(index))

            try:
                shutil.move(nest, outdir + 'trial_{0}/'.format(index))
            except:
                extension = nest.split('/')[-1]
                while os.path.exists(outdir + 'trial_{0}/'.format(index) + extension):
                    index += 1

                if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                    os.makedirs(outdir + 'trial_{0}/'.format(index))

                shutil.move(nest, outdir + 'trial_{0}/'.format(index))

    remove_empty_dirs(dirname)
    

def get_stats(dirname):
    """ Get missing runs for all trials """
    hidden_dims = [32, 64, 128, 256, 512]
    batch_sizes = [128, 256, 512, 1024]
    learning_rates = [2e-1, 2e-2, 2e-3]

    filenames, hyperparams = [], []

    for (lr, hdim, bsize) in itertools.product(*[learning_rates, hidden_dims, batch_sizes]):
        hyperparam = (lr * min(batch_sizes)/bsize, hdim, bsize)
        filename = 'results_{0}.json'.format("_".join([str(i) for i in hyperparam]))
        filenames.append(filename)
        hyperparams.append((str(format_e(lr)), str(hdim), str(bsize)))
    
    TODO = []
    for file in os.listdir(dirname):
        if '.DS_Store' in file:
            continue

        print(file, len(os.listdir(dirname + file)))
        idx = 0
        try:
            for f in os.listdir(dirname + file):
                if '.DS_Store' in f:
                    continue

                files = os.listdir(dirname + file + '/' + f)
                length = len(files)
                print(f, length)

                if length >= 60:
                    idx += 1            
                else:
                    missing = [hyperparams[idx] for idx, item in enumerate(filenames) if item not in files]
                    TODO.extend(missing)

            print('{0}/20'.format(idx))
            print('\n')
        except NotADirectoryError:
            files = os.listdir(dirname + file)
            missing = [hyperparams[idx] for idx, item in enumerate(filenames) if item not in files]
            TODO.extend(missing)
            
        
    return TODO

In [ ]:
dirname = '/Users/sob/Desktop/apple/64_dims_100000_samples/'
outdir = dirname
for idx, file in enumerate(os.listdir(dirname)):

    if '.DS_Store' in file:
        continue

    for nest in crawl_directory(dirname + file):

        index = 1

        if 'dims' not in nest.split('/')[6]:
            outdir = '/'.join(nest.split('/')[:6]) + '/'# + nest.split('/')[8:9]) + '/'
        else:
            # Uncomment the + for mixture
            outdir = dirname + nest.split('/')[6]
        
        print(outdir)
            
#        # Initialize directory
        if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
            os.makedirs(outdir + 'trial_{0}/'.format(index))

        try:
            shutil.move(nest, outdir + 'trial_{0}/'.format(index))
        except:
            extension = nest.split('/')[-1]
            while os.path.exists(outdir + 'trial_{0}/'.format(index) + extension):
                index += 1

            if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                os.makedirs(outdir + 'trial_{0}/'.format(index))

            shutil.move(nest, outdir + 'trial_{0}/'.format(index))

remove_empty_dirs(dirname)

In [ ]:
""" MOVE HYPERTUNING RESULTS TO BEST FOLDER """
import tqdm
best_path = '/Users/sob/Desktop/gan_results/best/multivariate/64_dims_100000_samples/'
dirname = '/Users/sob/Desktop/gan_results/hypertuning/multivariate/64_dims_100000_samples/'
if not os.path.exists(best_path):
    os.makedirs(best_path)

files = os.listdir(dirname)
files = [f for f in files if f != '.DS_Store']
for idx, f in tqdm.tqdm_notebook(enumerate(files)):
    
    optimal = get_best_performance_mnist(dirname + f + '/')
    if len(os.listdir(dirname + f + '/')) < 60:
        print(f, len(os.listdir(dirname + f + '/')))
    
    with open(best_path + '/trial_{0}.json'.format(idx+1), 'w') as outfile:
        json.dump(optimal, outfile)